In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/MNIST_MoNet')

In [3]:
#import graph, coarsening, utils
%load_ext autoreload
%autoreload 1
%aimport graph
%aimport coarsening
%aimport utils

import tensorflow as tf
import time, shutil
import numpy as np
import os, collections, sklearn
import scipy.sparse as sp
import scipy
import matplotlib.pyplot as plt
import networkx as nx
import cv2
%matplotlib inline

In [0]:
#Definition of some flags useful later in the code

flags = tf.app.flags
FLAGS = flags.FLAGS
# Graphs.
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_integer('number_edges', 8, 'Graph: minimum number of edges per vertex.')
flags.DEFINE_string('metric', 'euclidean', 'Graph: similarity measure (between features).')
flags.DEFINE_bool('normalized_laplacian', True, 'Graph Laplacian: normalized.')
# Directories.
flags.DEFINE_string('dir_data', 'data_mnist', 'Directory to store data.')

In [5]:
#Here we proceed at computing the original grid where the images live and the various coarsening that are applied
#for each level


def get_rho(dist, idx):
    """Return the adjacency matrix of a 
    0.graph."""
    M, k = dist.shape
    assert M, k == idx.shape
    assert dist.min() >= 0

    # Weight matrix.
    I = np.arange(0, M).repeat(k)
    J = idx.reshape(M*k)
    V = dist.reshape(M*k)
    rhos = scipy.sparse.coo_matrix((V, (I, J)), shape=(M, M))

    # No self-connections.
    rhos.setdiag(0)

    # Non-directed graph.
    bigger = rhos.T > rhos
    rhos = rhos - rhos.multiply(bigger) + rhos.T.multiply(bigger)

    assert rhos.nnz % 2 == 0
    assert np.abs(rhos - rhos.T).mean() < 1e-10
    assert type(rhos) is scipy.sparse.csr.csr_matrix
    return rhos
  
  
def get_theta(angle, idx):
    """Return the adjacency matrix of a kNN graph."""
    M, k = angle.shape
    assert M, k == idx.shape

    # Weight matrix.
    I = np.arange(0, M).repeat(k)
    J = idx.reshape(M*k)
    V = angle.reshape(M*k)
    theta = scipy.sparse.coo_matrix((V, (I, J)), shape=(M, M))

    # No self-connections.
    theta.setdiag(0)

    # Non-directed graph.
    bigger = theta.T > theta
    theta = theta - theta.multiply(bigger) + theta.T.multiply(bigger)

    assert theta.nnz % 2 == 0
    assert np.abs(theta - theta.T).mean() < 1e-10
    assert type(theta) is scipy.sparse.csr.csr_matrix
    return theta
  
  
def grid_graph(m):
    z = graph.grid(m)
    dist, angle, idx = graph.distance_sklearn_metrics(z, k=FLAGS.number_edges, metric=FLAGS.metric) 
    #dist contains the distance of the 8 nearest neighbors for each node sorted in ascending order
    #idx contains the indexes of the 8 nearest for each node sorted in ascending order by distance

    rhos = get_rho(dist, idx)
    rhos.setdiag(0.0)
    rhos = rhos.tocoo()

    theta = get_theta(angle, idx)
    theta.setdiag(0.0)
    theta = theta.tocoo()
    return rhos, theta
  

def get_laplacian(m):
    z = graph.grid(m)
    dist, angle, idx = graph.distance_sklearn_metrics(z, k=FLAGS.number_edges, metric=FLAGS.metric) 
    #dist contains the distance of the 8 nearest neighbors for each node sorted in ascending order
    #idx contains the indexes of the 8 nearest for each node sorted in ascending order by distance

    A = graph.adjacency(dist, idx)
    L = graph.laplacian(A, normalized=FLAGS.normalized_laplacian)
    L = L.tocoo()
    return L

pool_step = 2
new_dim = 56
np.random.seed(0)
rho1, theta1 = grid_graph(new_dim)
rho2, theta2 = grid_graph(new_dim//pool_step)
rho3, theta3 = grid_graph(new_dim//pool_step**2)
Rhos = []
Rhos.append(rho1)
Rhos.append(rho2)
Rhos.append(rho3)
Thetas = []
Thetas.append(theta1)
Thetas.append(theta2)
Thetas.append(theta3)

L = []
L.append(get_laplacian(new_dim))
L.append(get_laplacian(new_dim//pool_step))
L.append(get_laplacian(new_dim//pool_step**2))

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr

In [6]:
#loading of MNIST dataset

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(FLAGS.dir_data, one_hot=False)

train_data = mnist.train.images.astype(np.float32)
val_data = mnist.validation.images.astype(np.float32) #the first 5K samples of the training dataset 
                                                      #are used for validation
test_data = mnist.test.images.astype(np.float32)
train_labels = mnist.train.labels
val_labels = mnist.validation.labels
test_labels = mnist.test.labels

new_train_data = np.zeros((train_data.shape[0], new_dim*new_dim))
new_val_data = np.zeros((val_data.shape[0], new_dim*new_dim))
new_test_data = np.zeros((test_data.shape[0], new_dim*new_dim))
for i in range(train_data.shape[0]):
    im = train_data[i,:].reshape(28,28)
    im = cv2.resize(im, (new_dim,new_dim), interpolation = cv2.INTER_LINEAR)
    new_train_data[i,:] = im.reshape(1,-1)
for i in range(val_data.shape[0]):
    im = val_data[i,:].reshape(28,28)
    im = cv2.resize(im, (new_dim,new_dim), interpolation = cv2.INTER_LINEAR)
    new_val_data[i,:] = im.reshape(1,-1)
for i in range(test_data.shape[0]):
    im = test_data[i,:].reshape(28,28)
    im = cv2.resize(im, (new_dim,new_dim), interpolation = cv2.INTER_LINEAR)
    new_test_data[i,:] = im.reshape(1,-1)

t_start = time.time()
train_data = new_train_data
val_data = new_val_data
test_data = new_test_data
print('Execution time: {:.2f}s'.format(time.time() - t_start))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data_mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data_mnist/train-labels-idx1-ubyte.gz
Extracting data_mnist/t10k-images-idx3-ubyte.gz
Extracting data_mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Execution time: 0.00s


In [0]:
class MoNet:
    """
    The neural network model.
    """
    
    #Helper functions used for constructing the model
    def _weight_variable(self, shape, regularization=True): 
        """Initializer for the weights"""
        
        initial = tf.truncated_normal_initializer(0, 0.1)
        var = tf.get_variable('weights', shape, tf.float32, initializer=initial)
        if regularization: #append the loss of the current variable to the regularization term 
            self.regularizers.append(tf.nn.l2_loss(var))
        return var
    
    def _bias_variable(self, shape, regularization=True):
        """Initializer for the bias"""
        
        initial = tf.constant_initializer(0.1)
        var = tf.get_variable('bias', shape, tf.float32, initializer=initial)
        if regularization:
            self.regularizers.append(tf.nn.l2_loss(var))
        return var
    def _mu_variable(self, shape, regularization=True): 
        """Initializer for the weights"""
        
        initial = tf.random_uniform_initializer(0,0.1)
        var = tf.get_variable('gaussian_mu', shape, tf.float32, initializer=initial)
        if regularization: #append the loss of the current variable to the regularization term 
            self.regularizers.append(tf.nn.l2_loss(var))
        return var  
    def _sigma_variable(self, shape, regularization=True): 
        """Initializer for the weights"""
        
        initial = tf.random_uniform_initializer(0,1)
        var = tf.get_variable('gaussian_sigma', shape, tf.float32, initializer=initial)
        if regularization: #append the loss of the current variable to the regularization term 
            self.regularizers.append(tf.nn.l2_loss(var))
        return var 

    def frobenius_norm(self, tensor): 
        """Computes the frobenius norm for a given tensor"""
        
        square_tensor = tf.square(tensor)
        tensor_sum = tf.reduce_sum(square_tensor)
        frobenius_norm = tf.sqrt(tensor_sum)
        return frobenius_norm
    
    
    def count_no_weights(self):
        total_parameters = 0
        for variable in tf.trainable_variables():
            # shape is an array of tf.Dimension
            shape = variable.get_shape()
            variable_parameters = 1
            for dim in shape:
                variable_parameters *= dim.value
            total_parameters += variable_parameters
        print('#weights in the model: %d' % (total_parameters,))

    
    #Modules used by the graph convolutional network
    def MoConv(self, x, L, Rho, Theta, Fout, n_gaussian): 
        """Applies Monet over the graph (i.e. it makes a spatial convolution)"""
        #Fout is the number of output features
        #Rho is pseudo-coordinates
        #n_gaussian is dimensionality of the extracted patch
        
        N, M, Fin = x.get_shape()  # N is the number of images
                                   # M the number of vertices in the images
                                   # Fin the number of features
        N, M, Fin = int(N), int(M), int(Fin)
        
        gaussian_filter = self.gaussianlayer(L, Rho, Theta, n_gaussian) #(M*n_gaussian, M)
        W = self._weight_variable([n_gaussian*Fin, Fout], regularization=False) #(n_gaussian*Fin, Fout)
        x = tf.transpose(x,[0,2,1]) 
        x = tf.reshape(x,[N*Fin, M])
        x = tf.transpose(x,[1,0]) #(M, N*Fin)
        gaussian_feats = tf.matmul(gaussian_filter, x) #(M*n_gaussian, N*Fin)
        gaussian_feats = tf.transpose(gaussian_feats, [1,0])
        gaussian_feats = tf.reshape(gaussian_feats, [N, Fin,M*n_gaussian])
        gaussian_feats = tf.transpose(gaussian_feats, [0,2,1]) 
        gaussian_feats = tf.reshape(gaussian_feats, [N, M, Fin*n_gaussian])
        gaussian_feats = tf.reshape(gaussian_feats, [N*M, Fin*n_gaussian])
        output = tf.matmul(gaussian_feats,W) #(N*M, Fout)
        output = tf.reshape(output, [N, M, Fout])
        return output
    
    def gaussianlayer(self, L, Rho, Theta, n_gaussian):
        #Rho is pseudo-coordinates = rho and theta
        #n_gaussian is dimensionality of the extracted patch (should be 2?)
        
        M, _ = Rho.shape #(each row corresponds to 1 node, containing the neighborhood of the node)
        M = int(M)
        mask = tf.sparse.SparseTensor(L.indices, [1.0]*(L.values.shape[0]), [M,M])
        mask = tf.sparse.to_dense(mask)
        mask = tf.expand_dims(mask,2)
        with tf.variable_scope('Rho'):
            Rho = tf.sparse.to_dense(Rho)
            Rho = tf.expand_dims(Rho, 2)
            #initialize mu and sigma
            rho_mu = tf.constant(0.0, shape=[1, 1, n_gaussian])
            rho_sigma = self._sigma_variable([1, 1, n_gaussian], regularization=True)
            #compute coefficience of MoNet patch operator
            g_weight_rho = tf.exp((Rho - rho_mu)**2*(-0.5/rho_sigma**2))
            #mask
            g_weight_rho = g_weight_rho * mask 
        with tf.variable_scope('Theta'):
            Theta = tf.sparse.to_dense(Theta)
            Theta = tf.expand_dims(Theta, 2)
            #initialize mu and sigma
            theta_mu = tf.constant(0.0, shape=[1, 1, n_gaussian])
            theta_sigma = self._sigma_variable([1, 1, n_gaussian], regularization=True)
            #compute coefficience of MoNet patch operator
            g_weight_theta = tf.exp((Theta - theta_mu)**2*(-0.5/theta_sigma**2))
            #mask
            g_weight_theta = g_weight_theta * mask 
        #normolize
        g_weight = g_weight_theta * g_weight_rho
        g_weight = g_weight/tf.reduce_sum(g_weight,1,keepdims=True)
        #reshape
        g_weight = tf.transpose(g_weight,(1,0,2))
        g_weight = tf.reshape(g_weight,(M, M*n_gaussian))
        return tf.transpose(g_weight,(1,0))
    
    def b1relu(self, x):
        """Applies bias and ReLU. One bias per filter."""
        N, M, F = x.get_shape()
        b = self._bias_variable([1, 1, int(F)], regularization=False)
        return tf.nn.relu(x + b) #add the bias to the convolutive layer


    def mpool1(self, x, p):
        """Max pooling of size p. Should be a power of 2 (this is possible thanks to the reordering we previously did)."""
        if p > 1:
            x = tf.expand_dims(x, 3)  # shape = N x M x F x 1
            x = tf.nn.max_pool(x, ksize=[1,p,1,1], strides=[1,p,1,1], padding='SAME')
            return tf.squeeze(x, [3])  # shape = N x M/p x F
        else:
            return x

    def fc(self, x, Mout, relu=True):
        """Fully connected layer with Mout features."""
        N, Min = x.get_shape()
        W = self._weight_variable([int(Min), Mout], regularization=True)
        b = self._bias_variable([Mout], regularization=True)
        x = tf.matmul(x, W) + b
        return tf.nn.relu(x) if relu else x
    
    #function used for extracting the result of our model
    def _inference(self, x, dropout): #definition of the model
        
        # Graph convolutional layers.
        x = tf.expand_dims(x, 2)  # N x M x F=1
        rec_size = [new_dim, new_dim//pool_step, new_dim//pool_step**2]
        for i in range(len(self.p)):
            with tf.variable_scope('cgconv{}'.format(i+1)):
                with tf.name_scope('filter'):
                    x = self.MoConv(x, self.L[i], self.Rhos[i], self.Thetas[i], self.F[i], self.K[i])
                with tf.name_scope('bias_relu'):
                    x = self.b1relu(x)
                with tf.name_scope('pooling'):
                    N, M, F = x.get_shape()
                    x = tf.reshape(x, [N, rec_size[i],rec_size[i], F])
                    x = tf.contrib.layers.instance_norm(x)
                    x = tf.nn.max_pool(x, ksize=[1,self.p[i],self.p[i],1], strides=[1,self.p[i],self.p[i],1], padding='SAME')
                    x = tf.reshape(x, [N, -1, F])
         
        # Fully connected hidden layers.
        N, M, F = x.get_shape()
        x = tf.reshape(x, [int(N), int(M*F)])  # N x M
        if self.M:
            for i,M in enumerate(self.M[:-1]): #apply a fully connected layer for each layer defined in M
                                              #(we discard the last value in M since it contains the number of classes we have
                                              #to predict)
                with tf.variable_scope('fc{}'.format(i+1)):
                    x = self.fc(x, M)
                    x = tf.nn.dropout(x, dropout)
            
            # Logits linear layer, i.e. softmax without normalization.
            with tf.variable_scope('logits'):
                x = self.fc(x, self.M[-1], relu=False)
        return x
    
    def convert_coo_to_sparse_tensor(self, L):
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data.astype('float32'), L.shape)
        L = tf.sparse_reorder(L)
        return L
    
    def __init__(self, p, K, F, M, M_0, batch_size, L, Rhos, Thetas,
                 decay_steps, decay_rate, learning_rate=1e-4, momentum=0.9, regularization=5e-4, clip_norm=1e1,
                 idx_gpu = '/gpu:0'):
        self.regularizers = list() #list of regularization l2 loss for multiple variables
        
        self.p = p #dimensions of the pooling layers
        self.K = K #List of polynomial orders, i.e. filter sizes or number of hops
        self.F = F #Number of features of convolutional layers
        
        self.M = M #Number of neurons in fully connected layers
        
        self.M_0 = M_0 #number of elements in the first graph 
        
        self.batch_size = batch_size
        
        #definition of some learning parameters
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate
        self.learning_rate = learning_rate
        self.regularization = regularization
        
        with tf.Graph().as_default() as g:
                self.graph = g
                tf.set_random_seed(0)
                with tf.device(idx_gpu):
                        #definition of placeholders
                        self.Rhos = [self.convert_coo_to_sparse_tensor(Rho) for Rho in Rhos]
                        self.Thetas = [self.convert_coo_to_sparse_tensor(Theta) for Theta in Thetas]
                        self.L = [self.convert_coo_to_sparse_tensor(c_L) for c_L in L]
                        self.ph_data = tf.placeholder(tf.float32, (self.batch_size, M_0), 'data')
                        self.ph_labels = tf.placeholder(tf.int32, (self.batch_size), 'labels')
                        self.ph_dropout = tf.placeholder(tf.float32, (), 'dropout')
                    
                        #Model construction
                        self.logits = self._inference(self.ph_data, self.ph_dropout)
                        
                        #Definition of the loss function
                        with tf.name_scope('loss'):
                            self.cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, 
                                                                                                labels=self.ph_labels)
                            self.cross_entropy = tf.reduce_mean(self.cross_entropy)
                        with tf.name_scope('regularization'):
                            if self.M:
                                self.regularization *= tf.add_n(self.regularizers)
                        self.loss = self.cross_entropy + self.regularization
                        
                        # Create a session for running Ops on the Graph.
                        config = tf.ConfigProto(allow_soft_placement = True)
                        config.gpu_options.allow_growth = True
                        self.session = tf.Session(config=config)
                        #Solver Definition
                        with tf.name_scope('training'):
                            # Learning rate.
                            self.global_step = tf.Variable(0, name='global_step', trainable=False) #used for counting how many iterations we have done
                            if decay_rate != 1: #applies an exponential decay of the lr wrt the number of iterations done
                                learning_rate = tf.train.exponential_decay(
                                        learning_rate, self.global_step, decay_steps, decay_rate, staircase=True)
                            # Optimizer.
                            if momentum == 0:
                                optimizer = tf.train.AdamOptimizer(learning_rate)
                            else: #applies momentum for increasing the robustness of the gradient 
                                optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
                            #grads = optimizer.compute_gradients(self.loss)
                            tvars = tf.trainable_variables()
                            #grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), clip_norm)
                            variable_names = [v.name for v in tvars]
                            variables_to_restore = {v.name.split(":")[0]: v for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)}
                            skip_pretrained_var = []
                            variables_to_restore = {v: variables_to_restore[v] for v in variables_to_restore if not any(x in v for x in skip_pretrained_var)}
                            if variables_to_restore:
                                saver_pre_trained = tf.train.Saver(var_list=variables_to_restore)
                                ckpt = tf.train.get_checkpoint_state('./gdrive/My Drive/MNIST_MoNet/model_save/')
                                if ckpt != None:
                                  saver_pre_trained.restore(self.session, ckpt.model_checkpoint_path) 
                                  print('Relodaing parameters')
                            train_vars = [v for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES) if any(x in v.name for x in skip_pretrained_var)]

                            grads, variables = zip(*optimizer.compute_gradients(self.loss, var_list=tvars))
                            grads, _ = tf.clip_by_global_norm(grads, clip_norm)
                            self.op_gradients = optimizer.apply_gradients(zip(grads, variables), 
                                                                          global_step=self.global_step)
                            
                        #Computation of the norm gradients (useful for debugging)
                        self.var_grad = tf.gradients(self.loss, tf.trainable_variables())
                        self.norm_grad = self.frobenius_norm(tf.concat([tf.reshape(g, [-1]) for g in self.var_grad], 0))

                        #Extraction of the predictions and computation of accuracy
                        self.predictions = tf.cast(tf.argmax(self.logits, dimension=1), tf.int32)
                        self.accuracy = 100 * tf.contrib.metrics.accuracy(self.predictions, self.ph_labels)
        
                        uninitialized_vars = []
                        for var in tf.all_variables():
                            try:
                                self.session.run(var)
                            except tf.errors.FailedPreconditionError:
                                uninitialized_vars.append(var)

                        init_new_vars_op = tf.initialize_variables(uninitialized_vars)
                        self.session.run(init_new_vars_op) 
                        self.saver = tf.train.Saver(tf.global_variables())
                        self.count_no_weights()

In [0]:
#Convolutional parameters
p = [pool_step, pool_step, new_dim//pool_step**2]   #Dimensions of the pooling layers
K = [18, 18, 18] #List of gaussians distribution
F = [32, 32, 64] #Number of features of convolutional layers

#FC parameters
C = max(mnist.train.labels) + 1 #Number of classes we have
M = [512, C] #Number of neurons in fully connected layers. If there is no FC, set M = []

#Solver parameters
batch_size = 100
decay_steps = mnist.train.num_examples / batch_size #number of steps to do before decreasing the learning rate
decay_rate = 0.95 #how much decreasing the learning rate
learning_rate = 0.02
momentum = 0.9
regularization = 5e-4

#Definition of keep probabilities for dropout layers
dropout_training = 0.5
dropout_val_test = 1.0

In [9]:
#Construction of the learning obj
M_0 = Rhos[0].shape[0] #number of elements in the first graph
learning_obj = MoNet(p, K, F, M, M_0, batch_size, L, Rhos, Thetas,
                       decay_steps, decay_rate,
                       learning_rate=learning_rate, regularization=regularization,
                       momentum=momentum)

#definition of overall number of training iterations and validation frequency
num_iter_val = 550
num_total_iter_training = 16501

num_iter = 0

list_training_loss = list()
list_training_norm_grad = list()
list_val_accuracy = list()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Please use tf.global_variables instead.
Instructions for updating:
Use `tf.variables_initializer` instead.
#weights in the model: 94774


In [10]:
#training and validation
indices = collections.deque() #queue that will contain a permutation of the training indexes
for k in range(num_iter, num_total_iter_training):
    
    #Construction of the training batch
    if len(indices) < batch_size: # Be sure to have used all the samples before using one a second time.
        indices.extend(np.random.permutation(train_data.shape[0])) #reinitialize the queue of indices
    idx = [indices.popleft() for i in range(batch_size)] #extract the current batch of samples
    
    #data extraction
    batch_data, batch_labels = train_data[idx,:], train_labels[idx] 
    
    feed_dict = {learning_obj.ph_data: batch_data, 
                 learning_obj.ph_labels: batch_labels, 
                 learning_obj.ph_dropout: dropout_training}
    
    #Training
    tic = time.time()
    _, current_training_loss, norm_grad = learning_obj.session.run([learning_obj.op_gradients, 
                                                                    learning_obj.loss, 
                                                                    learning_obj.norm_grad], feed_dict = feed_dict) 
    training_time = time.time() - tic
    
    list_training_loss.append(current_training_loss)
    list_training_norm_grad.append(norm_grad)
    
    if (np.mod(num_iter, num_iter_val)==0): #validation
        msg = "[TRN] iter = %03i, cost = %3.2e, |grad| = %.2e (%3.2es)" \
                    % (num_iter, list_training_loss[-1], list_training_norm_grad[-1], training_time)
        print(msg)
        
        #Validation Code
        tic = time.time()
        val_accuracy = 0
        for begin in range(0, val_data.shape[0], batch_size):
            end = begin + batch_size
            end = min([end, val_data.shape[0]])
            
            #data extraction
            batch_data = np.zeros((end-begin, val_data.shape[1]))
            batch_data = val_data[begin:end,:]
            batch_labels = np.zeros(batch_size)
            batch_labels[:end-begin] = val_labels[begin:end]
            
            feed_dict = {learning_obj.ph_data: batch_data, 
                         learning_obj.ph_labels: batch_labels,
                         learning_obj.ph_dropout: dropout_val_test}
            
            batch_accuracy = learning_obj.session.run(learning_obj.accuracy, feed_dict)
            val_accuracy += batch_accuracy*batch_data.shape[0]
        val_accuracy = val_accuracy/val_data.shape[0]
        val_time = time.time() - tic
        msg = "[VAL] iter = %03i, acc = %4.2f (%3.2es)" % (num_iter, val_accuracy, val_time)
        print(msg)
        learning_obj.saver.save(learning_obj.session, "./gdrive/My Drive/MNIST_MoNet/model_save/model.ckpt", global_step=learning_obj.global_step)
    num_iter += 1

[TRN] iter = 000, cost = 1.20e+01, |grad| = 1.75e+02 (4.69e+00s)
[VAL] iter = 000, acc = 13.90 (9.63e+00s)


KeyboardInterrupt: ignored